In [28]:
# importamos las librerías que necesitamos
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Evaluar linealidad de las relaciones entre las variables
# ------------------------------------------------------------------------------
from scipy.stats import shapiro, kstest
from itertools import combinations


# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#from scipy.stats import shapiro, kstest, poisson, chisquare, ttest_ind, levene, bartlett, sem, ppf
import scipy.stats as stats

#  Fase 3: Evaluación de Diferencias en Reservas de Vuelos por Nivel Educativo

###  Utilizando un conjunto de datos que hemos compartido, se busca evaluar si existen diferencias significativas en el número de vuelos reservados según el nivel educativo de los clientes. Para ello, los pasos que deberas seguir son:

## 1. Preparación de Datos:
### Filtra el conjunto de datos para incluir únicamente las columnas relevantes: 'Flights Booked' y 'Education'.

In [29]:
# abrimos el csv
df = pd.read_csv("datos_limpio_unidos.csv")
df.head()

,Unnamed: 0,numero_cliente,ano,mes,vuelos_reservados,vuelos_con_acompanantes,total_vuelos,distancia,puntos_acumulados,puntos_canjeados,costo_en_dolares_de_los_puntos_canjeados,provincia,ciudad,codigo_postal,genero,educacion,salario,estado_civil,tarjeta_lealtad,valoracion_cliente,tipo_inscripcion,ano_inscripcion,mes_inscripcion,ano_cancelacion,mes_cancelacion,estado_cliente,columna_float
0,0,100018,2017,1,3,0,3,1521,152.0,0,0,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,999,99,Activo,NaN
1,1,100102,2017,1,10,4,14,2030,203.0,0,0,Ontario,Toronto,M1R 4K3,Male,College,73455.0,Single,Nova,2887.74,Standard,2013,3,999,99,Activo,NaN
2,2,100140,2017,1,6,0,6,1200,120.0,0,0,British Columbia,Dawson Creek,U5I 4F1,Female,College,73455.0,Divorced,Nova,2838.07,Standard,2016,7,999,99,Activo,NaN
3,3,100214,2017,1,0,0,0,0,0.0,0,0,British Columbia,Vancouver,V5R 1W3,Male,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,999,99,Activo,NaN
4,4,100272,2017,1,0,0,0,0,0.0,0,0,Ontario,Toronto,P1L 8X8,Female,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,999,99,Activo,NaN


In [30]:
# filtramos 
df_filtrado = df[["numero_cliente",'vuelos_reservados', 'educacion']]
df_filtrado.head()

,numero_cliente,vuelos_reservados,educacion
0,100018,3,Bachelor
1,100102,10,College
2,100140,6,College
3,100214,0,Bachelor
4,100272,0,Bachelor


## 2. Análisis Descriptivo:
### Agrupa los datos por nivel educativo y calcula estadísticas descriptivas básicas (como el promedio, la desviación estandar, los percentiles) del número de vuelos reservados para cada grupo.

In [31]:
# calculos estadisticos
df_filtrado.groupby("educacion")["vuelos_reservados"].describe()

,count,mean,std,min,25%,50%,75%,max
educacion,,,,,,,,
Bachelor,253752.0,4.091093,5.216995,0.0,0.0,1.0,8.0,21.0
College,102672.0,4.153012,5.242136,0.0,0.0,1.0,8.0,21.0
Doctor,17856.0,4.146281,5.250093,0.0,0.0,1.0,8.0,21.0
High School or Below,19008.0,4.155777,5.234551,0.0,0.0,1.0,8.0,21.0
Master,12336.0,4.184014,5.210294,0.0,0.0,1.0,8.0,21.0


Cosas que vemos:

* La media de reserva de vuelos es similar entre los distintos niveles de educacion.


* Parece que el patrón de reserva de vuelos no cambia en función de la educación.


* La mayoria de nuestros clientes han realizado Bachillerato

# 3. Prueba Estadística:
## Realiza una prueba de A/B testing para determinar si existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos

H0 : No existen diferencias en la reserva de vuelos segun tu nivel educactivo.

H1 : Si existen diferencias en la reserva de vuelos según el nivel educatico.

In [32]:
# calculamos la media por grupo
df_media = df_filtrado.groupby(['numero_cliente', "educacion"])["vuelos_reservados"].mean().reset_index()
df_media.head()

,numero_cliente,educacion,vuelos_reservados
0,100018,Bachelor,6.541667
1,100102,College,7.208333
2,100140,College,6.333333
3,100214,Bachelor,3.291667
4,100272,Bachelor,5.291667


In [33]:
conteo_df = df.groupby("educacion")["numero_cliente"].count()
porcentaje_educacion_df = (conteo_df / conteo_df.sum()) * 100
porcentaje_educacion_df

educacion
Bachelor                62.558428
College                 25.312112
Doctor                   4.402106
High School or Below     4.686113
Master                   3.041240
Name: numero_cliente, dtype: float64

La mayoria de nuestros clientes son de estudios Bachelor por lo que vamos hacer 3 tres grupos, bajo medio y alto. Para hacer dos grupos,mas o menos con la misma proporcion vamos a coger los extremos. Doctor para evaluar a los de alta y High School para lo de baja ya que tienen un porcentaje parecido

In [34]:
# Funicón que clasifica en función de los estudios
def clasificar_grupo(educacion):
    if educacion in ['Doctor']:
        return 'Alto'
    elif educacion in ['Bachelor', 'College', "'Master'"]:
        return 'Medio'
    else:
        return 'Bajo'



In [35]:
df_media['grupo_educativo'] = df_media['educacion'].apply(clasificar_grupo)
df_media.head()

,numero_cliente,educacion,vuelos_reservados,grupo_educativo
0,100018,Bachelor,6.541667,Medio
1,100102,College,7.208333,Medio
2,100140,College,6.333333,Medio
3,100214,Bachelor,3.291667,Medio
4,100272,Bachelor,5.291667,Medio


In [42]:
# Filtrar los grupos 1 (Alto) y 3 (Bajo)
grupo_a = df_media[df_media['grupo_educativo'] == 'Alto']
grupo_b = df_media[df_media['grupo_educativo'] == 'Bajo']


In [39]:
# Realizar una prueba de normalidad (usando la prueba de Shapiro-Wilk)
p_value = shapiro(df_filtrado['vuelos_reservados']).pvalue

alpha = 0.05 # nivel significancia = 5% -> alpha = 0.05
if p_value > alpha:
    print("Los datos se ajustan a una distribución normal (p-value =", p_value, ")")
else:
    print("Los datos no se ajustan a una distribución normal (p-value =", p_value, ")")

Los datos no se ajustan a una distribución normal (p-value = 7.090914983682426e-165 )


Tras esta prueba, los resultados obtenidos indican que las muestras no siguen una distribución normal.
Por lo tanto, necesitamos utilizar una prueba no paramétrica para comparar los grupos. En este caso, utilizaremos el test de Mann Whitney. Este test es una prueba estadística no paramétrica utilizada para comparar dos muestras independientes y determinar si provienen de poblaciones con medianas iguales o diferentes. Esta prueba es apropiada cuando los datos no siguen una distribución normal y se utiliza para evaluar si hay diferencias significativas entre dos grupos.

In [38]:
def test_man_whitney(dataframe, columna_metrica, grupo_control, grupo_test, columna_grupos):
    """
    Realiza la prueba de Mann-Whitney U para comparar las medianas de una métrica entre dos grupos en un DataFrame dado.

    Parámetros:
    - dataframe (DataFrame): El DataFrame que contiene los datos.
    - columna_metrica (str): El nombre de la columna que representa la métrica a comparar entre los grupos.
    - grupo_control (str): El nombre del grupo de control en la columna especificada por columna_grupos.
    - grupo_test (str): El nombre del grupo de test en la columna especificada por columna_grupos.
    - columna_grupos (str): El nombre de la columna que contiene la información de los grupos.

    Returns:
    No devuelve nada directamente, pero imprime en la consola si las medianas son diferentes o iguales para la métrica.
    Se utiliza la prueba de Mann-Whitney U para evaluar si hay diferencias significativas entre los grupos.
    """
    # Filtramos el DataFrame para quedarnos solo con los datos del grupo de control
    control = dataframe[dataframe[columna_grupos] == grupo_control]
    
    # Filtramos el DataFrame para quedarnos solo con los datos del grupo de prueba
    test = dataframe[dataframe[columna_grupos] == grupo_test]
    
    # Extraemos la columna de métrica para cada grupo
    metrica_control = control[columna_metrica]
    metrica_test = test[columna_metrica]
    
    # Aplicamos el estadístico de Mann-Whitney U
    u_statistic, p_value = stats.mannwhitneyu(metrica_control, metrica_test)
    
    # Imprimimos el resultado
    if p_value < 0.05:
        print(f"Para la métrica {columna_metrica}, las medianas son diferentes (p-value = {p_value}).")
    else:
        print(f"Para la métrica {columna_metrica}, las medianas son iguales (p-value = {p_value}).")
        
# Llamar a la función con una sola columna de métrica
test_man_whitney(df_media, "vuelos_reservados", "Alto", "Bajo", "grupo_educativo")

Para la métrica vuelos_reservados, las medianas son iguales (p-value = 0.683387305664922).


Por lo tanto, no existe una diferencia significativa entre los dos grupos, ya que los p-valores son mayores que 0.05.
Conclusión: No hay diferencia en el numero de vuelos reservados en funcion de los niveles educativos Bajo y Alto